In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

# Load Datasets

In [2]:
def load_data(file_path):
    A = np.loadtxt(file_path)
    X = A[:, :9]    # Input features
    y = A[:, 9:]    # Output labels
    return X, y

In [3]:
# Load all 3 datasets
X_final, y_final = load_data('data/tictac_final.txt')
y_final = y_final[:, 0]

X_single, y_single = load_data('data/tictac_single.txt')
y_single = y_single[:, 0]

# SVM Classifier

In [4]:
def svm_train(X, y):
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    # Define and train model
    model = LinearSVC(dual="auto")
    model.fit(X_train, y_train)

    # Get cross validation accuracy
    val_accuracy = cross_val_score(model, X_train, y_train, cv=10, scoring="accuracy")
    val_accuracy = np.mean(val_accuracy)

    # Get test accuracy
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    # Get confusion matrix
    confusion_mtrx = confusion_matrix(y_test, y_pred, normalize="true")

    return val_accuracy, test_accuracy, confusion_mtrx


In [5]:
def print_results(val_accuracy, test_accuracy, confusion_mtrx, dataset_name):
    print(f"Performance of Linear SVM Classification on {dataset_name}:")
    print("Cross Validation Accuracy = ", val_accuracy)
    print("Test Accuracy = ", test_accuracy)
    print("Confusion Matrix:")
    print(confusion_mtrx.round(decimals=3)) # Round to 3 decimal places

## Final Dataset

In [6]:
val_acc_final, test_acc_final, confusion_mtrx_final = svm_train(X_final, y_final)
print_results(val_acc_final, test_acc_final, confusion_mtrx_final, "Final Dataset")

Performance of Linear SVM Classification on Final Dataset:
Cross Validation Accuracy =  0.9869617224880383
Test Accuracy =  0.96875
Confusion Matrix:
[[0.91 0.09]
 [0.   1.  ]]


## Single Dataset

In [7]:
val_acc_single, test_acc_single, confusion_mtrx_single = svm_train(X_single, y_single)
print_results(val_acc_single, test_acc_single, confusion_mtrx_single, "Single Dataset")

Performance of Linear SVM Classification on Single Dataset:
Cross Validation Accuracy =  0.23549618320610688
Test Accuracy =  0.2402745995423341
Confusion Matrix:
[[0.938 0.    0.003 0.    0.059 0.    0.    0.    0.   ]
 [0.94  0.006 0.03  0.    0.024 0.    0.    0.    0.   ]
 [0.979 0.    0.005 0.    0.016 0.    0.    0.    0.   ]
 [0.923 0.    0.043 0.    0.034 0.    0.    0.    0.   ]
 [0.926 0.005 0.02  0.    0.05  0.    0.    0.    0.   ]
 [0.934 0.    0.039 0.    0.026 0.    0.    0.    0.   ]
 [0.909 0.01  0.02  0.    0.061 0.    0.    0.    0.   ]
 [0.84  0.    0.02  0.    0.14  0.    0.    0.    0.   ]
 [0.989 0.011 0.    0.    0.    0.    0.    0.    0.   ]]
